#### Hàm Extract Date Name

In [5]:
import os
import re
import json



def extract_information(
    input_dir="D:/2023-playwright/text-from-html-data-test", 
    output_dir="D:/2023-playwright/shorten/test-2023/text-from-html-data", 
):
    parent_dir = os.path.basename(os.path.dirname(input_dir))
    year_match = re.search(r"\d{4}", parent_dir)
    if year_match:
        base_year = int(year_match.group())
    else:
        print("Không tìm thấy năm trong tên thư mục!")
        return

    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"Input directory not found: {input_dir}")
    os.makedirs(output_dir, exist_ok=True)

    # File tổng hợp
    aggregate_file_path = "D:/2023-playwright/shorten_aggregate_file.txt"

    # Khởi tạo từ khóa và regex
    date_name_keywords = ["submission", "deadline", "paper", "notification", "acceptance", "camera", "due", "ready", "final", "author"]
    special_keywords = [r"\$", r"http", r"volume"]
    month_keywords = [
        "Jan", "Feb", "Mar", "Apr", "Jun", "May", "Jul", "Aug", "Sep", "Sept", "Oct", "Nov", "Dec",
        "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"
    ]

    # Regex patterns
    date_name_pattern = re.compile(r"(\b" + r"\b|\b".join(date_name_keywords) + r"\b)", re.IGNORECASE)
    month_pattern = re.compile(r"(\b" + r"\b|\b".join(month_keywords) + r"\b)", re.IGNORECASE)
    time_pattern = re.compile(r"\b\d{1,2}:\d{2}\b")
    special_pattern = re.compile(r"(" + r"|".join(special_keywords) + r")", re.IGNORECASE)
    four_digit_pattern = re.compile(r"(\d{4})")
    non_four_digit_pattern = re.compile(r"(?<!\d)\d{1,3}(?!\d)")
    two_number_valid_or_four_number_pattern = re.compile(r"(?<!\d)(0?[1-9]|[12][0-9]|3[01])(?!\d)|(\d{4})")


    # Hàm kiểm tra số lớn hơn 31
    def contains_large_number(line):
        numbers = [int(num) for num in non_four_digit_pattern.findall(line)]
        return any(num > 31 for num in numbers)

    # Hàm kiểm tra dòng hợp lệ
    def is_valid_line(line):
        year_matches = four_digit_pattern.findall(line)
        contains_valid_year = not year_matches or all(base_year - 1 <= int(year) <= base_year + 1 for year in year_matches)

        return (
            not contains_large_number(line) and
            not time_pattern.search(line) and
            not special_pattern.search(line) and
            contains_valid_year
        )

    # Hàm thêm ngữ cảnh xung quanh một dòng
    def add_context_lines(lines, idx, added_lines, context_range=2):
        context_buffer = []
        for i in range(-context_range, context_range + 1):
            neighbor_idx = idx + i
            if 0 <= neighbor_idx < len(lines) and (lines[neighbor_idx], neighbor_idx) not in added_lines:
                context_buffer.append(lines[neighbor_idx])
                added_lines.add((lines[neighbor_idx], neighbor_idx))
        return context_buffer

    # Xử lý các file
    with open(aggregate_file_path, "w", encoding="utf-8") as aggregate_file:
        for filename in os.listdir(input_dir):
            file_path = os.path.join(input_dir, filename)
            if not os.path.isfile(file_path):
                continue


            # Trích xuất từ khóa từ tên file (phần trước dấu gạch dưới cuối cùng)
            file_keyword = "_".join(filename.rsplit('_', 1)[0].split('_'))

            with open(file_path, "r", encoding="utf-8") as file:
                lines = file.readlines()

            final_lines = []
            added_lines = set()

            for idx, line in enumerate(lines):
                contains_date_keyword = date_name_pattern.search(line)
                contains_day_and_month = month_pattern.search(line) and two_number_valid_or_four_number_pattern.search(line)

                contains_geo_entity = re.search(
                    fr"(\b({base_year}|{file_keyword})\b)|"
                    fr"(\b(in|at)\b.*\b(conference|event|{file_keyword})\b|"
                    fr"\b(conference|event|{file_keyword})\b.*\b(in|at)\b)", 
                    line, 
                    re.IGNORECASE
                )

                if any([contains_date_keyword, contains_day_and_month, contains_geo_entity]) and is_valid_line(line):
                    context_lines = add_context_lines(lines, idx, added_lines)
                    final_lines.extend(context_lines)
            

            # Lọc các dòng không chứa số lớn hơn 31
            final_lines = [
                line for line in final_lines
                if is_valid_line(line)  # Loại bỏ dòng nếu chứa số lớn hơn 31
            ]

            # Loại bỏ dòng trống và ghi kết quả
            final_lines = [line for line in final_lines if line.strip()]
            if final_lines:
                # output_path = os.path.join(output_dir, filename)
                # with open(output_path, "w", encoding="utf-8") as output_file:
                #     output_file.write("".join(final_lines))

                aggregate_file.write(f"File: {filename}\n")
                aggregate_file.write("".join(final_lines) + "\n\n")

    print("Aggregate file created at:", aggregate_file_path)

extract_information()


Aggregate file created at: D:/2023-playwright/shorten_aggregate_file.txt


In [ ]:
def aggregate_all_files(
    input_dir="D:/2023-playwright/text-from-html-data-test", 
    output_file="D:/2023-playwright/shorten/test-2023/text-from-html-data"
    ):
    """
    Gộp nội dung của tất cả các file trong thư mục input_dir vào một file tổng hợp.
    """

    # Mở file tổng hợp để ghi
    with open(output_file, "w", encoding="utf-8") as aggregate_file:
        total_lines = 0

        # Duyệt qua từng file trong thư mục input
        for filename in os.listdir(input_dir):
            file_path = os.path.join(input_dir, filename)

            # Kiểm tra nếu là file
            if os.path.isfile(file_path):
                with open(file_path, "r", encoding="utf-8") as file:
                    lines = file.readlines()

                # Loại bỏ các dòng trống
                non_empty_lines = [line for line in lines if line.strip() != ""]

                # Đếm số dòng không trống và ghi vào file tổng hợp
                line_count = len(non_empty_lines)
                total_lines += line_count

                
                aggregate_file.write(f"File: {filename}\n")
                aggregate_file.write("".join(lines) + "\n\n")
                print(f"Đã gộp file {filename} với {line_count} dòng.")

        print(f"Tổng số dòng đã gộp từ tất cả các file: {total_lines}")

    print(f"File tổng hợp được tạo tại: {output_file}")

aggregate_all_files()

PermissionError: [Errno 13] Permission denied: 'D:/2023-playwright/shorten/test-2023/text-from-html-data'